#  ASSIGNMENT 1 : Text BASED SENTIMENT ANALYSIS



*   Loading text


In [28]:
from google.colab import drive
import json
drive.mount('/content/drive')
%cd /content/drive/MyDrive/reviews/

# Data Loading & Preprocessing
file_path = 'Cell_Phones_and_Accessories_5.json'
reviews_list = []

# Load JSON file
with open(file_path, 'r') as json_file:
    for review_line in json_file:
        data = json.loads(review_line)
        reviews_list.append(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/reviews


DATA PREPROCESSING



*   Removal of punctuation
*   Removal of stopwords
*   Removal of unecessary columns
*   Removal of stopwords



In [30]:
# Retain necessary columns
keys_to_keep = ['reviewerID', 'asin', 'reviewText', 'overall', 'summary', 'reviewTime']
filtered_reviews = [{key: review[key] for key in keys_to_keep if key in review} for review in reviews_list]

# Convert text data to lowercase
text_columns = ['reviewText', 'summary']
for review in filtered_reviews:
    for col in text_columns:
        review[col] = review[col].lower()

# Load stopwords
%cd /content/drive/MyDrive/stopword_folder/
stopwords_file = 'NLTK\'s list of english stopwords'
with open(stopwords_file, 'r') as file:
    stopwords = set(file.read().splitlines())

# Remove stopwords and punctuation
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation
    return ' '.join([word for word in text.split() if word not in stopwords])

for review in filtered_reviews:
    for col in text_columns:
        review[col] = clean_text(review[col])



/content/drive/MyDrive/stopword_folder


THEMATIC ANALYSIS


In [31]:
import re
from collections import Counter
#Calculating frequency for each word in positive and negative reiews:
 # Setting the threshold if overall rating is greater than 4.0 ,it will be considered as good review and less that 2.0 will be considered as negative review
 # we have used counter function to count the number of positive and negative reviews to get the count of words in each review text
positive_reviews = []
negative_reviews = []

for review in filtered_reviews:
    if review['overall'] >= 4.0:
        positive_reviews.append(review)
    elif review['overall'] <= 2.0:
        negative_reviews.append(review)

def word_frequency(reviews):
    counter = Counter()
    for review in reviews:
        counter.update(review['reviewText'].split())
    return counter

positive_word_count = word_frequency(positive_reviews)
negative_word_count = word_frequency(negative_reviews)

def identify_keywords(word_count, threshold=4):
    return {word: count for word, count in word_count.items() if count >= threshold}

positive_keywords = identify_keywords(positive_word_count)
negative_keywords = identify_keywords(negative_word_count)



SEMANTIC ANALYSIS

In [32]:
word_weights = {"good": 0.65, "amazing": 0.7, "bad": -0.5, "worst": -0.7, "excellent": 0.8, "terrible": -0.8}

def sentiment_analysis(text, weights):
    words = text.split()
    score = sum(weights.get(word, 0) for word in words)
    return score

def categorize_sentiment(score, threshold=0.5):
    if score > threshold:
        return "Positive"
    elif score < -threshold:
        return "Negative"
    else:
        return "Neutral"

%cd /content/drive/MyDrive/

with open("sentiments.txt", "w", encoding="utf-8") as file:
    for review in filtered_reviews:
        review_text = review['reviewText']
        score = sentiment_analysis(review_text, word_weights)
        sentiment = categorize_sentiment(score)
        file.write(f"Review ID: {review['reviewerID']}, Sentiment: {sentiment}\n")
        file.write(f"Review Text: {review_text}\n")
        file.write("*" * 50 + "\n")


/content/drive/MyDrive
